### Libraries

In [1]:
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
plt.ion() 
import numpy as np
import pathlib 
import mne
from mne import Epochs 
from mne.time_frequency import psd_array_multitaper
import mplcursors 
import scipy
from datetime import datetime
import pandas as pd
from matplotlib.widgets import Cursor
from scipy.signal import welch
import re
import os
import glob
import csv
matplotlib.use('Qt5Agg')
pd.set_option('display.float_format', '{:.0f}'.format)
from mne.preprocessing import ICA
from mne.io.constants import FIFF

### Functions

In [2]:
def import_sorted_fif_files(directory):
    """
    This function lists all .fif files in a specified directory, sorts them 
    based on the numeric value after the 'P' in the filename, and imports the 
    files into a list using MNE's read_raw_fif function.

    Parameters:
    directory (str): The path to the directory containing the .fif files.

    Returns:
    fif_tasks (list): A list of MNE Raw objects, each representing an imported 
    .fif file, in the order sorted by the numeric part after 'P' in the filename.
    """
    
    # List all files in the specified directory and filter for those ending with '.fif'
    fif_files = [f for f in os.listdir(directory) if f.endswith('.fif')]

    # Function to extract the numeric part after 'P' in the filename for sorting
    def extract_number(filename):
        """
        Extracts the number that follows 'P' and precedes '_T' in the filename.
        If no match is found, returns infinity to ensure that such files are sorted last.

        Parameters:
        filename (str): The filename to extract the number from.

        Returns:
        int or float: The extracted number or infinity if no match is found.
        """
        match = re.search(r'P(\d+)_T\d+\.fif', filename)
        return int(match.group(1)) if match else float('inf')
    
    # Sort the files by the numeric value extracted from the filename
    fif_files.sort(key=extract_number)

    # Import the .fif files in the sorted order and store them in a list
    fif_tasks = [mne.io.read_raw_fif(os.path.join(directory, f), preload=True) for f in fif_files]

    return fif_tasks


In [3]:
from datetime import datetime

def get_unix_times(fif_file):
    """
    Extracts the UNIX start and end time of an EEG recording from a .fif file.

    Parameters:
    ----------
    fif_file : mne.io.Raw
        A cleaned EEG signal stored in MNE's .fif format.

    Returns:
    -------
    eeg_start_unix : int
        The UNIX timestamp representing the start of the EEG recording, adjusted for the GMT to IST time offset (5 hours 30 minutes).
    
    eeg_end_unix : int
        The UNIX timestamp representing the end of the EEG recording, calculated using the signal duration and adjusted start time.
    """
    
    # Get the measurement start time from the .fif file metadata (meas_date).
    meas_date = fif_file.info['meas_date']
    
    # Handle different types of meas_date:
    # - If it's a tuple, take the first element and convert it to a timestamp.
    # - If it's already a datetime object, use its timestamp.
    # - If neither, raise a ValueError as the format is unrecognized.
    if isinstance(meas_date, tuple):
        meas_date_timestamp = meas_date[0].timestamp()  # meas_date[0] is a datetime object
    elif isinstance(meas_date, datetime):
        meas_date_timestamp = meas_date.timestamp()
    else:
        raise ValueError("meas_date is not in a recognized format. Expected a tuple or datetime object.")

    # Time correction factor to adjust for the GMT to IST (Indian Standard Time) offset, which is 5 hours and 30 minutes (19800 seconds).
    time_correction = 19800  # 5 hours 30 minutes = 5 * 3600 + 30 * 60 = 19800 seconds
    eeg_start_unix = int(meas_date_timestamp - time_correction)

    # Calculate the duration of the signal in seconds:
    # - sfreq: Sampling frequency (Hz) indicates how many data points are recorded per second.
    # - fif_file.times: An array representing time points of the recording.
    # The total duration is the length of the time array divided by the sampling frequency.
    sfreq = fif_file.info['sfreq']
    signal_duration_secs = int(len(fif_file.times) / sfreq)

    # Compute the UNIX timestamp for the end of the EEG recording by adding the signal duration to the start time.
    eeg_end_unix = int(eeg_start_unix + signal_duration_secs)

    return eeg_start_unix, eeg_end_unix

In [4]:
def bandpower(data, sf, band, method='welch', window_sec=None, relative=False):
    """Compute the average power of the signal x in a specific frequency band.
    Requires MNE-Python >= 0.14.

    Parameters
    ----------
    data : 1d-array
      Input signal in the time-domain.
    sf : float
      Sampling frequency of the data.
    band : list
      Lower and upper frequencies of the band of interest.
    method : string
      Periodogram method: 'welch' or 'multitaper'
    window_sec : float
      Length of each window in seconds. Useful only if method == 'welch'.
      If None, window_sec = (1 / min(band)) * 2.
    relative : boolean
      If True, return the relative power (= divided by the total power of the signal).
      If False (default), return the absolute power.

    Return
    ------
    bp : float
      Absolute or relative band power.
    freq_res : float
      Frequency resolution of the bandpower calculation
    """
    from scipy.signal import welch
    from scipy.integrate import simps
    from mne.time_frequency import psd_array_multitaper

    band = np.asarray(band)
    low, high = band

    # Compute the modified periodogram (Welch)
    if method == 'welch':
        if window_sec is not None:
            nperseg = window_sec * sf
        else:
            nperseg = (2 / low) * sf

        freqs, psd = welch(data, sf, nperseg=nperseg)

    elif method == 'multitaper':
        psd, freqs = psd_array_multitaper(data, sf, adaptive=True,
                                          normalization='full', n_jobs=1, remove_dc= True, max_iter = 1024, verbose=0)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    return bp, freq_res

In [5]:
def import_segments(directory):
    """
    Imports segment start and end times from a CSV file and stores them in two separate lists.

    Parameters:
    ----------
    directory : str
        The file path to the CSV file containing segment start and end times.

    Returns:
    -------
    seg_start_time : list of str
        A list containing the start times of the segments, read from the first column of the CSV.
    
    seg_end_time : list of str
        A list containing the end times of the segments, read from the second column of the CSV.
    
    Notes:
    -----
    - The CSV file is expected to have at least two columns where:
        - The first column contains segment start times.
        - The second column contains segment end times.
    - The first row of the CSV is assumed to be a header and is skipped during import.
    """

    # Initialize the lists to store segment start and end times
    seg_start_time = []
    seg_end_time = []

    # Open the CSV file in read mode
    with open(directory, mode='r') as file:
        csv_reader = csv.reader(file)
        
        # Skip the header row (assumed to be the first row)
        next(csv_reader)
        
        # Iterate through the remaining rows in the CSV
        for row in csv_reader:
            # Append the start time from the first column to seg_start_time
            seg_start_time.append(row[0])  # First column: Start times
            
            # Append the end time from the second column to seg_end_time
            seg_end_time.append(row[1])    # Second column: End times

    # Optionally print the arrays to verify their contents
    print("Segment Start Times:", seg_start_time)
    print("Segment End Times:", seg_end_time)

    # Return the start and end times as two separate lists
    return seg_start_time, seg_end_time


In [6]:
def make_time_array(fif_file, segment_csv_path):
    """
    Processes segment start and end times from a CSV file, aligns them with the 
    UNIX start time of an EEG recording in a .fif file, and calculates the relative 
    start times and other time-related metrics.

    Parameters:
    ----------
    fif_file : mne.io.Raw
        The EEG recording stored in MNE's .fif format.

    segment_csv_path : str
        The path to the CSV file containing the segment start and end times.

    Returns:
    -------
    rel_time_sec : np.array
        An array of relative segment start times, measured from the first segment's start time, in seconds.
    
    seg_start_array : list of int
        A list of absolute segment start times in UNIX time, extracted and converted from the CSV.
    
    seg_end_array : list of int
        A list of absolute segment end times in UNIX time, extracted and converted from the CSV.
    
    t_seg_min : int
        The offset of the first segment start time relative to the beginning of the EEG recording (in seconds).
    
    t_seg_max : int
        The offset of the last segment end time relative to the beginning of the EEG recording (in seconds).
    
    Notes:
    -----
    - The function prints time-related details of the EEG recording and segments.
    - Assumes that the first segment's start time is the reference point for calculating relative times.
    """

    # Import segment start and end times from the CSV file
    seg_start, seg_end = import_segments(segment_csv_path)
    
    # Count the number of segments
    segment_count = len(seg_start)
    
    # Get the UNIX start and end time of the EEG recording
    unix_start_reading, unix_end_reading = get_unix_times(fif_file)
    
    # Calculate the total duration of the EEG recording in seconds
    total_reading_time = unix_end_reading - unix_start_reading
    
    # Convert segment start and end times from strings to integers
    seg_start_array = [int(m) for m in seg_start]
    seg_end_array = [int(m) for m in seg_end]
    
    # Use the first segment's start time as the reference point
    seg_start = seg_start_array[0]
    
    # Calculate relative start times for each segment, using the first segment's start time as the base
    seg_start_array_rel = [x - seg_start for x in seg_start_array]
    rel_time_sec = np.array(seg_start_array_rel)

    # Calculate the total duration of all segments
    total_segment_duration = seg_end_array[-1] - seg_start_array[0]
    
    # Calculate the offset of the first segment's start time relative to the start of the EEG recording
    offset = seg_start_array[0] - unix_start_reading
    
    # Calculate the minimum and maximum relative times for the segments
    t_seg_min = offset
    t_seg_max = offset + total_segment_duration
    
    # Print the time-related details of the EEG reading and segments
    print(f'Unix start time of the reading = {unix_start_reading}')
    print(f'Unix end time of the reading = {unix_end_reading}')
    print(f'Duration of signal = {total_reading_time} sec')
    print(f'No of segments: {segment_count}')
    print(f'Total duration of segments = {total_segment_duration}')
    print(f'Relative segment start times in seconds: {rel_time_sec}')
    print(f't_seg_min = {t_seg_min}')
    print(f't_seg_max = {t_seg_max}')
    print(f'Segments start at {t_seg_min} seconds and end at {t_seg_max} seconds')

    # Return the calculated values
    return rel_time_sec, seg_start_array, seg_end_array, t_seg_min, t_seg_max


In [7]:
def crop_segments(fif_file, rel_time_sec, t_seg_max):
    """
    Crops a given .fif file based on provided time segments and stores these cropped segments into a list.

    Parameters:
    -----------
    fif_file : mne.io.Raw
        Cleaned EEG reading to be segmented. This is typically one of the readings selected for protocol analysis.
    
    rel_time_sec : np.ndarray
        An array of relative start times for the segments (in seconds), where each value represents the time 
        from the start of the signal. The length of this array should be equal to the number of segments + 1.

    t_seg_max : int
        The maximum time (in seconds) to which the .fif file should be cropped.

    Returns:
    --------
    cropped_fif_segments : list of mne.io.Raw
        A list containing the cropped .fif segments, where each element is a cropped version of the original file.
    
    segmentwise_events : list of np.ndarray
        A list of event arrays generated for each segment, where each array contains events that are spaced at 
        a fixed interval (1-second duration).
    
    Notes:
    ------
    - This function assumes the sampling frequency is 256 Hz.
    - The .fif file is cropped at the start and end times specified by `rel_time_sec`.
    - An additional crop is performed for the final segment from the last relative time to the full file duration.
    """

    # Calculate the number of segments (subtract 1 since rel_time_sec has n+1 elements)
    segment_count = len(rel_time_sec) - 1
    
    # Create a copy of the fif_file to preserve the original data
    fif_temp_last = fif_file.copy()

    # Calculate the total duration of the .fif file in seconds, assuming a sampling frequency of 256 Hz
    fif_file_duration = int(len(fif_file.times) / 256)
    print(f'Duration of .fif file: {fif_file_duration} seconds')

    # Initialize lists to store the cropped .fif segments and segment-wise event arrays
    cropped_fif_segments = []
    segmentwise_events = []

    # Loop through each segment to crop the .fif file and generate events
    for i in range(segment_count):
        x = rel_time_sec[i]
        y = rel_time_sec[i + 1]
        print('-----')
        print(f'Segment {i} start time = {x} seconds')
        print(f'Segment {i} end time = {y} seconds')
        print('-----')

        # Copy the original .fif file and crop it based on the current segment's start and end times
        fif_temp = fif_file.copy()
        fif_temp_cropped = fif_temp.crop(tmin=x, tmax=y)
        
        # Create fixed-length events (duration 1 second) for the current segment
        seg_event = mne.make_fixed_length_events(fif_file, start=x, stop=y, duration=1.0)
        
        # Append the cropped segment and the events array to their respective lists
        cropped_fif_segments.append(fif_temp_cropped)
        segmentwise_events.append(seg_event)
        print(f'Segment {i} appended to cropped segments')
        print(f'Events array for segment {i} appended')

    # Handle the final segment, which extends from the last relative start time to the end of the .fif file
    fif_temp_cropped_last = fif_temp_last.crop(tmin=rel_time_sec[-1], tmax=fif_file_duration)
    cropped_fif_segments.append(fif_temp_cropped_last)
    
    # Create events for the last segment and append them to the list
    seg_event = mne.make_fixed_length_events(fif_file, start=rel_time_sec[-1], stop=fif_file_duration, duration=1.0)
    segmentwise_events.append(seg_event)
    print(f'Last segment appended to cropped segments')
    print(f'Events array for the last segment appended')

    # Return the list of cropped segments and the corresponding event arrays
    return cropped_fif_segments, segmentwise_events


In [8]:
def crop_copy(raw, start_unix, end_unix, t1, t2):
    """
    Crop a raw MNE object based on a time window specified by UNIX timestamps.

    This function takes a raw EEG/MEG recording, represented as an `mne.io.Raw` object, and crops it based on
    two provided UNIX timestamps (t1 and t2), which specify the start and end of the desired time window. 
    The timestamps are converted to relative times based on the start of the recording.

    Parameters:
    -----------
    raw : mne.io.Raw
        The raw MNE object containing the recording to be cropped. This object holds EEG or MEG data, along with
        associated metadata.
    
    start_unix : int
        The UNIX timestamp representing the start time of the recording. This is the time when the EEG/MEG recording
        began, and it is used to calculate the relative crop times.
    
    end_unix : int
        The UNIX timestamp representing the end time of the recording. This is included for completeness but is not 
        used in the cropping calculation. 
    
    t1 : int
        The seconds timestamp indicating the start of the desired crop window. This is the time at which the cropped 
        segment should begin.
    
    t2 : int
        The seconds timestamp indicating the end of the desired crop window. This is the time at which the cropped 
        segment should end.

    Returns:
    --------
    raw_temp : mne.io.Raw
        A new `mne.io.Raw` object that contains the cropped data. This object is a copy of the original `raw` object, 
        but restricted to the time window specified by `t1` and `t2`.

    Notes:
    ------
    - The cropping is based on relative times (in seconds) calculated from the start of the recording (start_unix).
    - If `t1` is earlier than the recording start, the crop window will begin from 0 seconds (the start of the recording).
    - If `t2` is later than the recording end, the crop window will extend only to the end of the available data.
    - This function returns a copy of the original `raw` object; the original remains unmodified.
    """

    # Convert the recording start time from UNIX timestamp to a datetime object
    recording_start_time = datetime.utcfromtimestamp(start_unix)

    # Convert UNIX timestamps t1 and t2 to datetime objects (start and end of the crop window)
    start_time = datetime.utcfromtimestamp(t1)
    end_time = datetime.utcfromtimestamp(t2)

    # Calculate the relative times (in seconds) from the start of the recording
    relative_start_time = (start_time - recording_start_time).total_seconds()
    relative_end_time = (end_time - recording_start_time).total_seconds()

    # Ensure the relative start time is not before the start of the recording
    relative_start_time = max(0, relative_start_time)
    
    # Ensure the relative end time does not exceed the length of the recording
    relative_end_time = min(raw.times[-1], relative_end_time)

    # Print out the calculated relative times for debugging
    print(f'Relative start time (seconds): {relative_start_time}')
    print(f'Relative end time (seconds): {relative_end_time}')

    # Copy the raw object to create a new instance for cropping
    raw_temp = raw.copy()

    # Crop the copied raw object using the calculated relative times
    raw_temp.crop(tmin=relative_start_time, tmax=relative_end_time)

    return raw_temp


In [9]:
def epoch_equally(rawf_8, epoch_duration): 
    # Create fixed-length events
    event_id = 1  # Assign a label to the events
    event_duration = epoch_duration  
    events = mne.make_fixed_length_events(rawf_8, id=event_id, duration=event_duration)

    # Define epoch parameters
    tmin = 0  # Start at the event
    tmax = epoch_duration # End 2 seconds after the event

    # Create epochs
    epochs_8 = mne.Epochs(rawf_8, events, event_id=event_id, tmin=tmin, tmax=tmax, reject_by_annotation= True, baseline=None, preload=True)
    return epochs_8
    

In [10]:
def concatenate_epochs_into_channels(epochs_8):
    
    epochs_8_data = epochs_8.get_data()
    n_epochs, n_channels, n_samples = epochs_8_data.shape

    # Initialize a list to store concatenated arrays for each channel
    concatenated_channels = []

    # Loop through each channel
    for channel in range(n_channels):
        # Extract the first row of the current channel across all epochs
        concatenated = np.concatenate([epochs_8_data[epoch, channel, :] for epoch in range(n_epochs)])
        concatenated_channels.append(concatenated)

    # Convert to a NumPy array if needed
    concatenated_channels = np.array(concatenated_channels)
    
    af3_signal_array = concatenated_channels[0][:]
    f7_signal_array = concatenated_channels[1][:]
    f3_signal_array = concatenated_channels[2][:]
    fc5_signal_array = concatenated_channels[3][:]
    fc6_signal_array = concatenated_channels[4][:]
    f4_signal_array = concatenated_channels[5][:]
    f8_signal_array = concatenated_channels[6][:]
    af4_signal_array = concatenated_channels[7][:]

    eeg_channels_data = { 
        'AF3': af3_signal_array ,
        'F7': f7_signal_array ,
        'F3': f3_signal_array ,
        'FC5': fc5_signal_array ,
        'FC6': fc6_signal_array,
        'F4': f4_signal_array,
        'F8': f8_signal_array,
        'AF4': af4_signal_array,
    }

    return eeg_channels_data
      

In [11]:
def channelwise_abp(eeg_channels_data_dictionary):
    sf = 256
    bands = {
        "delta": [1, 4],
        "theta": [4, 8],
        "alpha": [8, 13],
        "beta": [13, 30],
        "brain": [1, 30],
        "full": [0, 128]
    }

    # Extract channel signals
    channels = ["AF3", "F7", "F3", "FC5", "FC6", "F4", "F8", "AF4"]
    all_abp_set = []

    for channel in channels:
        signal = eeg_channels_data_dictionary[channel]
        abp_set = []
        for band_name, band_range in bands.items():
            abp, _ = bandpower(signal, sf, band_range, method='multitaper', window_sec=None, relative=False)
            abp_set.append(abp)
        all_abp_set.append(abp_set)

    # Convert to NumPy array
    all_abp_set = np.array(all_abp_set)
    return all_abp_set


In [12]:
participant_list = ['P1',
 'P3',
 'P4',
 'P5',
 'P6',
 'P7',
 'P8',
 'P9',
'P10',                   
'P11',
'P12',
'P13',
'P14',
'P15',
'P16',
'P17',
'P18',
'P19',
'P20',
'P21',
'P22',
'P23',
'P24',
'P25',
'P26',
'P27',
'P28',
'P29',
'P30',
'P31',
'P32']

### Main Code

In [13]:
#Import details for fif files for all tasks
directory_fif_task1 = '/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1'
directory_fif_task2 = '/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 2'
directory_fif_task3 = '/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3'
fif_task_1 = import_sorted_fif_files(directory_fif_task1)
fif_task_2 = import_sorted_fif_files(directory_fif_task2)
fif_task_3 = import_sorted_fif_files(directory_fif_task3)

Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P1_T1.fif...
    Range : 0 ... 24831 =      0.000 ...    96.996 secs
Ready.


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/1529551201.py:37: RuntimeWarning: This filename (/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P1_T1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  fif_tasks = [mne.io.read_raw_fif(os.path.join(directory, f), preload=True) for f in fif_files]


Reading 0 ... 24831  =      0.000 ...    96.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P3_T1.fif...
    Range : 0 ... 22783 =      0.000 ...    88.996 secs
Ready.
Reading 0 ... 22783  =      0.000 ...    88.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P4_T1.fif...
    Range : 0 ... 20991 =      0.000 ...    81.996 secs
Ready.
Reading 0 ... 20991  =      0.000 ...    81.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P5_T1.fif...
    Range : 0 ... 27135 =      0.000 ...   105.996 secs
Ready.
Reading 0 ... 27135  =      0.000 ...   105.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P6_T1.fif...
    Range : 0 ... 27647 =      0.000 ...   107.996 secs
Ready.
Reading 0 ..

/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/1529551201.py:37: RuntimeWarning: This filename (/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P3_T1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  fif_tasks = [mne.io.read_raw_fif(os.path.join(directory, f), preload=True) for f in fif_files]
/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/1529551201.py:37: RuntimeWarning: This filename (/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P4_T1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  fif_tasks = [

Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P10_T1.fif...
    Range : 0 ... 22271 =      0.000 ...    86.996 secs
Ready.
Reading 0 ... 22271  =      0.000 ...    86.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P11_T1.fif...
    Range : 0 ... 17919 =      0.000 ...    69.996 secs
Ready.
Reading 0 ... 17919  =      0.000 ...    69.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P12_T1.fif...
    Range : 0 ... 23295 =      0.000 ...    90.996 secs
Ready.
Reading 0 ... 23295  =      0.000 ...    90.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 1/P13_T1.fif...
    Range : 0 ... 20735 =      0.000 ...    80.996 secs
Ready.
Reading 0 ... 20735  =      0.000 ...    80.996 secs...
Opening 

...

/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/1529551201.py:37: RuntimeWarning: This filename (/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P24_T3.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  fif_tasks = [mne.io.read_raw_fif(os.path.join(directory, f), preload=True) for f in fif_files]
/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/1529551201.py:37: RuntimeWarning: This filename (/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P25_T3.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  fif_tasks =

Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P28_T3.fif...
    Range : 0 ... 239359 =      0.000 ...   934.996 secs
Ready.
Reading 0 ... 239359  =      0.000 ...   934.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P29_T3.fif...
    Range : 0 ... 135167 =      0.000 ...   527.996 secs
Ready.
Reading 0 ... 135167  =      0.000 ...   527.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P30_T3.fif...
    Range : 0 ... 236543 =      0.000 ...   923.996 secs
Ready.
Reading 0 ... 236543  =      0.000 ...   923.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P31_T3.fif...


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/1529551201.py:37: RuntimeWarning: This filename (/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P28_T3.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  fif_tasks = [mne.io.read_raw_fif(os.path.join(directory, f), preload=True) for f in fif_files]
/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/1529551201.py:37: RuntimeWarning: This filename (/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P29_T3.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  fif_tasks =

    Range : 0 ... 89343 =      0.000 ...   348.996 secs
Ready.
Reading 0 ... 89343  =      0.000 ...   348.996 secs...
Opening raw data file /Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P32_T3.fif...
    Range : 0 ... 70143 =      0.000 ...   273.996 secs
Ready.
Reading 0 ... 70143  =      0.000 ...   273.996 secs...


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/1529551201.py:37: RuntimeWarning: This filename (/Users/apple/Documents/Research/04. EEG Research/Python_Analysis/ICA/Cleaned EEG Data/Task 3/P32_T3.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  fif_tasks = [mne.io.read_raw_fif(os.path.join(directory, f), preload=True) for f in fif_files]


In [14]:
# Cropping baseline signal from the full signal and storing in a participant-wise ist

i = 0
j = 0
k = 0

fif_1_baseline = []
fif_2_baseline = []
fif_3_baseline = []

for fif in fif_task_1 :
    fif_temp = fif_task_1[i].copy()
    t1 = 0
    t2 = 30
    fif_task_baseline = fif_temp.crop(tmin=t1, tmax=t2)
    fif_1_baseline.append(fif_task_baseline)
    i = i+1

for fif in fif_task_2 :
    fif_temp = fif_task_2[j].copy()
    t1 = 0
    t2 = 30
    fif_task_baseline = fif_temp.crop(tmin=t1, tmax=t2)
    fif_2_baseline.append(fif_task_baseline)
    j = j+1

for fif in fif_task_3 :
    fif_temp = fif_task_3[k].copy()
    t1 = 0
    t2 = 30
    fif_task_baseline = fif_temp.crop(tmin=t1, tmax=t2)
    fif_3_baseline.append(fif_task_baseline)
    k = k+1

In [15]:
# Cropping sketching signal from the full signal and storing in a participant-wise ist

i = 0
j = 0
k = 0

fif_1_sketching_signals = []
fif_2_sketching_signals = []
fif_3_sketching_signals = []

for fif in fif_task_1 :
    fif_temp = fif_task_1[i].copy()
    t1 = 30
    t2 = fif_temp.times[-1]
    fif_task_signal = fif_temp.crop(tmin=t1, tmax=t2)
    fif_1_sketching_signals.append(fif_task_signal)
    i = i+1

for fif in fif_task_2 :
    fif_temp = fif_task_2[j].copy()
    t1 = 30
    t2 = fif_temp.times[-1]
    fif_task_signal = fif_temp.crop(tmin=t1, tmax=t2)
    fif_2_sketching_signals.append(fif_task_signal)
    j = j+1

for fif in fif_task_3 :
    fif_temp = fif_task_3[k].copy()
    t1 = 30
    t2 = fif_temp.times[-1]
    fif_task_signal = fif_temp.crop(tmin=t1, tmax=t2)
    fif_3_sketching_signals.append(fif_task_signal)
    k = k+1

In [17]:
# fif_1_participantwise_abp = []
# fif_1_bl_participantwise_abp = []
# fif_1_participantwise_abp_df = []
# fif_1_bl_participantwise_abp_df = []

# fif_2_participantwise_abp = []
# fif_2_bl_participantwise_abp = []
# fif_2_participantwise_abp_df = []
# fif_2_bl_participantwise_abp_df = []

# fif_3_participantwise_abp = []
# fif_3_bl_participantwise_abp = []
# fif_3_participantwise_abp_df = []
# fif_3_bl_participantwise_abp_df = []

# epoch_duration_abp = 5


# j= 0
# for fif in fif_3_baseline:
#     bl_sketch_temp = fif_3_baseline[j]
#     bl_sketch_temp_epochs = epoch_equally(bl_sketch_temp,epoch_duration_abp)
#     bl_channelwise_data_temp = concatenate_epochs_into_channels(bl_sketch_temp_epochs)
#     bl_abp_list_temp = channelwise_abp(bl_channelwise_data_temp)
#     fif_3_bl_participantwise_abp.append(bl_abp_list_temp)

#     channel_names = np.array(["AF3", "F7", "F3", "FC5", "FC6", "F4", "F8", "AF4"])
#     bl_abp_df_list_temp = pd.DataFrame(bl_abp_list_temp, columns=['bl_delta_abp','bl_theta_abp','bl_alpha_abp','bl_beta_abp','bl_brain_abp','bl_full_abp'])
#     pd.options.display.float_format ='{:.2e}'.format
#     bl_abp_df_list_temp['Channel'] = channel_names
#     fif_3_bl_participantwise_abp_df.append(bl_abp_df_list_temp)
#     j=j+1

# i= 0
# for fif in fif_3_sketching_signals:
#     sketch_temp = fif_3_sketching_signals[i]
#     sketch_temp_epochs = epoch_equally(sketch_temp,epoch_duration_abp)
#     channelwise_data_temp = concatenate_epochs_into_channels(sketch_temp_epochs)
#     abp_list_temp = channelwise_abp(channelwise_data_temp)
#     fif_3_participantwise_abp.append(abp_list_temp)

#     channel_names = np.array(["AF3", "F7", "F3", "FC5", "FC6", "F4", "F8", "AF4"])
#     abp_df_list_temp = pd.DataFrame(abp_list_temp, columns=['delta_abp','theta_abp','alpha_abp','beta_abp','brain_abp','full_abp'])
#     pd.options.display.float_format ='{:.2e}'.format
#     abp_df_list_temp['Channel'] = channel_names
#     fif_3_participantwise_abp_df.append(abp_df_list_temp)
#     i=i+1

    

Not setting metadata
6 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 6 events and 1281 original time points ...
2 bad epochs dropped


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/662386645.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_8_data = epochs_8.get_data()


Not setting metadata
6 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 6 events and 1281 original time points ...
0 bad epochs dropped


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/662386645.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_8_data = epochs_8.get_data()


Not setting metadata
6 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 6 events and 1281 original time points ...
1 bad epochs dropped


...

/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/662386645.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_8_data = epochs_8.get_data()


Not setting metadata
63 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 63 events and 1281 original time points ...
7 bad epochs dropped


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/662386645.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_8_data = epochs_8.get_data()


Not setting metadata
48 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 48 events and 1281 original time points ...
1 bad epochs dropped


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/662386645.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_8_data = epochs_8.get_data()


In [88]:
# fif_2_participantwise_abp = []
# fif_2_bl_participantwise_abp = []
# fif_2_participantwise_abp_df = []
# fif_2_bl_participantwise_abp_df = []

# epoch_duration_abp = 5

# #BASELINE TASK 2
# j= 0
# for fif in fif_2_baseline:
#     bl_sketch_temp = fif_2_baseline[j]
#     bl_sketch_temp_epochs = epoch_equally(bl_sketch_temp,epoch_duration_abp)
#     bl_channelwise_data_temp = concatenate_epochs_into_channels(bl_sketch_temp_epochs)
#     bl_abp_list_temp = channelwise_abp(bl_channelwise_data_temp)
#     fif_2_bl_participantwise_abp.append(bl_abp_list_temp)

#     channel_names = np.array(["AF3", "F7", "F3", "FC5", "FC6", "F4", "F8", "AF4"])
#     bl_abp_df_list_temp = pd.DataFrame(bl_abp_list_temp, columns=['bl_delta_abp','bl_theta_abp','bl_alpha_abp','bl_beta_abp','bl_brain_abp','bl_full_abp'])
#     pd.options.display.float_format ='{:.2e}'.format
#     bl_abp_df_list_temp['Channel'] = channel_names
#     fif_2_bl_participantwise_abp_df.append(bl_abp_df_list_temp)
#     j=j+1

# #SKETCHING TASK 2
# i= 0
# for fif in fif_2_sketching_signals:
#     sketch_temp = fif_2_sketching_signals[i]
#     sketch_temp_epochs = epoch_equally(sketch_temp,epoch_duration_abp)
#     channelwise_data_temp = concatenate_epochs_into_channels(sketch_temp_epochs)
#     abp_list_temp = channelwise_abp(channelwise_data_temp)
#     fif_2_participantwise_abp.append(abp_list_temp)

#     channel_names = np.array(["AF3", "F7", "F3", "FC5", "FC6", "F4", "F8", "AF4"])
#     abp_df_list_temp = pd.DataFrame(abp_list_temp, columns=['delta_abp','theta_abp','alpha_abp','beta_abp','brain_abp','full_abp'])
#     pd.options.display.float_format ='{:.2e}'.format
#     abp_df_list_temp['Channel'] = channel_names
#     fif_2_participantwise_abp_df.append(abp_df_list_temp)
#     i=i+1

Not setting metadata
6 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 6 events and 1281 original time points ...
0 bad epochs dropped


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/662386645.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_8_data = epochs_8.get_data()


Not setting metadata
6 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 6 events and 1281 original time points ...
0 bad epochs dropped


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/662386645.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_8_data = epochs_8.get_data()


Not setting metadata
6 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 6 events and 1281 original time points ...
0 bad epochs dropped


...

/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/662386645.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_8_data = epochs_8.get_data()


Not setting metadata
41 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 41 events and 1281 original time points ...
0 bad epochs dropped


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/662386645.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_8_data = epochs_8.get_data()


Not setting metadata
69 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 69 events and 1281 original time points ...
0 bad epochs dropped


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/662386645.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_8_data = epochs_8.get_data()


In [100]:
fif_1_participantwise_abp = []
fif_1_bl_participantwise_abp = []
fif_1_participantwise_abp_df = []
fif_1_bl_participantwise_abp_df = []

epoch_duration_abp = 5

#BASELINE TASK 2
j= 0
for fif in fif_1_baseline:
    bl_sketch_temp = fif_1_baseline[j]
    bl_sketch_temp_epochs = epoch_equally(bl_sketch_temp,epoch_duration_abp)
    bl_channelwise_data_temp = concatenate_epochs_into_channels(bl_sketch_temp_epochs)
    bl_abp_list_temp = channelwise_abp(bl_channelwise_data_temp)
    fif_1_bl_participantwise_abp.append(bl_abp_list_temp)

    channel_names = np.array(["AF3", "F7", "F3", "FC5", "FC6", "F4", "F8", "AF4"])
    bl_abp_df_list_temp = pd.DataFrame(bl_abp_list_temp, columns=['bl_delta_abp','bl_theta_abp','bl_alpha_abp','bl_beta_abp','bl_brain_abp','bl_full_abp'])
    pd.options.display.float_format ='{:.2e}'.format
    bl_abp_df_list_temp['Channel'] = channel_names
    fif_1_bl_participantwise_abp_df.append(bl_abp_df_list_temp)
    j=j+1

#SKETCHING TASK 2
i= 0
for fif in fif_1_sketching_signals:
    sketch_temp = fif_1_sketching_signals[i]
    sketch_temp_epochs = epoch_equally(sketch_temp,epoch_duration_abp)
    channelwise_data_temp = concatenate_epochs_into_channels(sketch_temp_epochs)
    abp_list_temp = channelwise_abp(channelwise_data_temp)
    fif_1_participantwise_abp.append(abp_list_temp)

    channel_names = np.array(["AF3", "F7", "F3", "FC5", "FC6", "F4", "F8", "AF4"])
    abp_df_list_temp = pd.DataFrame(abp_list_temp, columns=['delta_abp','theta_abp','alpha_abp','beta_abp','brain_abp','full_abp'])
    pd.options.display.float_format ='{:.2e}'.format
    abp_df_list_temp['Channel'] = channel_names
    fif_1_participantwise_abp_df.append(abp_df_list_temp)
    i=i+1

Not setting metadata
6 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 6 events and 1281 original time points ...
0 bad epochs dropped


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/662386645.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_8_data = epochs_8.get_data()


Not setting metadata
6 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 6 events and 1281 original time points ...
0 bad epochs dropped


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/662386645.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_8_data = epochs_8.get_data()


Not setting metadata
6 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 6 events and 1281 original time points ...
0 bad epochs dropped


...

/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/662386645.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_8_data = epochs_8.get_data()


Not setting metadata
8 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 8 events and 1281 original time points ...
1 bad epochs dropped


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/662386645.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_8_data = epochs_8.get_data()


Not setting metadata
8 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 8 events and 1281 original time points ...
1 bad epochs dropped


/var/folders/jq/xg0f849x4mg5f7y1_tjxk66c0000gn/T/ipykernel_4946/662386645.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs_8_data = epochs_8.get_data()


In [22]:
# m=0
# concatenated_bl_abp_list =[]
# for file in fif_3_bl_participantwise_abp_df :
#     temp_df = pd.concat([fif_3_bl_participantwise_abp_df[m],fif_3_participantwise_abp_df[m]], axis = 1)
#     concatenated_bl_abp_list.append(temp_df)
#     m = m+1

In [92]:
# m=0
# fif_2_concatenated_bl_abp_list =[]
# for file in fif_2_bl_participantwise_abp_df :
#     temp_df = pd.concat([fif_2_bl_participantwise_abp_df[m],fif_2_participantwise_abp_df[m]], axis = 1)
#     fif_2_concatenated_bl_abp_list.append(temp_df)
#     m = m+1

In [101]:
m=0
fif_1_concatenated_bl_abp_list =[]
for file in fif_1_bl_participantwise_abp_df :
    temp_df = pd.concat([fif_1_bl_participantwise_abp_df[m],fif_1_participantwise_abp_df[m]], axis = 1)
    fif_1_concatenated_bl_abp_list.append(temp_df)
    m = m+1

In [90]:
# for calculating relative powers :

fif_2_bl_participantwise_abp


[array([[3.07568029e-11, 2.01804649e-11, 8.68959677e-11, 1.55140158e-11,
         1.54016989e-10, 1.62366386e-10],
        [9.04305891e-12, 5.95528016e-12, 2.57839172e-11, 4.54341989e-12,
         4.55218596e-11, 4.79607458e-11],
        [2.21847417e-11, 1.44783945e-11, 6.25701280e-11, 1.10412657e-11,
         1.10702130e-10, 1.18895454e-10],
        [1.94625838e-11, 1.25795136e-11, 5.29906059e-11, 9.95891281e-12,
         9.54177096e-11, 1.00989133e-10],
        [1.12628336e-11, 7.42727593e-12, 3.22238220e-11, 5.64987797e-12,
         5.68076898e-11, 5.98421524e-11],
        [6.17828404e-11, 4.00294037e-11, 1.69215854e-10, 3.15453246e-11,
         3.03925735e-10, 3.21434602e-10],
        [2.30366673e-11, 1.52012824e-11, 6.60166879e-11, 1.15474220e-11,
         1.16300381e-10, 1.22506317e-10],
        [3.29593763e-11, 2.17279096e-11, 9.42210774e-11, 1.65399849e-11,
         1.66162381e-10, 1.75043742e-10]]),
 array([[4.43173715e-11, 1.12988981e-11, 1.49158466e-11, 1.19797556e-11,
     

In [36]:
dfc_temp_np = concatenated_bl_abp_list[0].to_numpy()

In [40]:
dfc_temp_np[0][0]

3.0797532665834947e-11

In [54]:
# p=0
# dfc_r_list = []

# for df in concatenated_bl_abp_list : 
#     dfc_temp = concatenated_bl_abp_list[p]
#     dfc_temp_np = concatenated_bl_abp_list[p].to_numpy()
    
#     bl_r_brain_delta=[]
#     bl_r_brain_theta=[]
#     bl_r_brain_alpha=[]
#     bl_r_brain_beta=[]
    
#     r_brain_delta=[]
#     r_brain_theta=[]
#     r_brain_alpha=[]
#     r_brain_beta=[]
    
#     o =0
#     for channel in dfc_temp_np :
#         bl_r_brain_delta_temp = (dfc_temp_np[o][0]/ dfc_temp_np[o][4])*100
#         bl_r_brain_delta.append(bl_r_brain_delta_temp)
#         bl_r_brain_theta_temp = (dfc_temp_np[o][1]/ dfc_temp_np[o][4])*100
#         bl_r_brain_theta.append(bl_r_brain_theta_temp)
#         bl_r_brain_alpha_temp = (dfc_temp_np[o][2]/ dfc_temp_np[o][4])*100
#         bl_r_brain_alpha.append(bl_r_brain_alpha_temp)
#         bl_r_brain_beta_temp = (dfc_temp_np[o][3]/ dfc_temp_np[o][4])*100
#         bl_r_brain_beta.append(bl_r_brain_beta_temp)
    
#         r_brain_delta_temp = (dfc_temp_np[o][7]/ dfc_temp_np[o][11])*100
#         r_brain_delta.append(r_brain_delta_temp)
#         r_brain_theta_temp = (dfc_temp_np[o][8]/ dfc_temp_np[o][11])*100
#         r_brain_theta.append(r_brain_theta_temp)
#         r_brain_alpha_temp = (dfc_temp_np[o][9]/ dfc_temp_np[o][11])*100
#         r_brain_alpha.append(r_brain_alpha_temp)
#         r_brain_beta_temp = (dfc_temp_np[o][10]/ dfc_temp_np[o][11])*100
#         r_brain_beta.append(r_brain_beta_temp)
#         o=o+1

#     dfc_temp['bl_r_brain_delta'] = bl_r_brain_delta
#     dfc_temp['bl_r_brain_theta'] = bl_r_brain_theta
#     dfc_temp['bl_r_brain_alpha'] = bl_r_brain_alpha
#     dfc_temp['bl_r_brain_beta'] = bl_r_brain_beta
#     dfc_temp['r_brain_delta'] = r_brain_delta
#     dfc_temp['r_brain_theta'] = r_brain_theta
#     dfc_temp['r_brain_alpha'] = r_brain_alpha
#     dfc_temp['r_brain_beta'] = r_brain_beta
#     dfc_r_list.append(dfc_temp)
#     p=p+1


In [93]:
# p=0
# fif_2_dfc_r_list = []

# for df in fif_2_concatenated_bl_abp_list : 
#     dfc_temp = fif_2_concatenated_bl_abp_list[p]
#     dfc_temp_np = fif_2_concatenated_bl_abp_list[p].to_numpy()
    
#     bl_r_brain_delta=[]
#     bl_r_brain_theta=[]
#     bl_r_brain_alpha=[]
#     bl_r_brain_beta=[]
    
#     r_brain_delta=[]
#     r_brain_theta=[]
#     r_brain_alpha=[]
#     r_brain_beta=[]
    
#     o =0
#     for channel in dfc_temp_np :
#         bl_r_brain_delta_temp = (dfc_temp_np[o][0]/ dfc_temp_np[o][4])*100
#         bl_r_brain_delta.append(bl_r_brain_delta_temp)
#         bl_r_brain_theta_temp = (dfc_temp_np[o][1]/ dfc_temp_np[o][4])*100
#         bl_r_brain_theta.append(bl_r_brain_theta_temp)
#         bl_r_brain_alpha_temp = (dfc_temp_np[o][2]/ dfc_temp_np[o][4])*100
#         bl_r_brain_alpha.append(bl_r_brain_alpha_temp)
#         bl_r_brain_beta_temp = (dfc_temp_np[o][3]/ dfc_temp_np[o][4])*100
#         bl_r_brain_beta.append(bl_r_brain_beta_temp)
    
#         r_brain_delta_temp = (dfc_temp_np[o][7]/ dfc_temp_np[o][11])*100
#         r_brain_delta.append(r_brain_delta_temp)
#         r_brain_theta_temp = (dfc_temp_np[o][8]/ dfc_temp_np[o][11])*100
#         r_brain_theta.append(r_brain_theta_temp)
#         r_brain_alpha_temp = (dfc_temp_np[o][9]/ dfc_temp_np[o][11])*100
#         r_brain_alpha.append(r_brain_alpha_temp)
#         r_brain_beta_temp = (dfc_temp_np[o][10]/ dfc_temp_np[o][11])*100
#         r_brain_beta.append(r_brain_beta_temp)
#         o=o+1

#     dfc_temp['bl_r_brain_delta'] = bl_r_brain_delta
#     dfc_temp['bl_r_brain_theta'] = bl_r_brain_theta
#     dfc_temp['bl_r_brain_alpha'] = bl_r_brain_alpha
#     dfc_temp['bl_r_brain_beta'] = bl_r_brain_beta
#     dfc_temp['r_brain_delta'] = r_brain_delta
#     dfc_temp['r_brain_theta'] = r_brain_theta
#     dfc_temp['r_brain_alpha'] = r_brain_alpha
#     dfc_temp['r_brain_beta'] = r_brain_beta
#     fif_2_dfc_r_list.append(dfc_temp)
#     p=p+1


In [102]:
p=0
fif_1_dfc_r_list = []

for df in fif_1_concatenated_bl_abp_list : 
    dfc_temp = fif_1_concatenated_bl_abp_list[p]
    dfc_temp_np = fif_1_concatenated_bl_abp_list[p].to_numpy()
    
    bl_r_brain_delta=[]
    bl_r_brain_theta=[]
    bl_r_brain_alpha=[]
    bl_r_brain_beta=[]
    
    r_brain_delta=[]
    r_brain_theta=[]
    r_brain_alpha=[]
    r_brain_beta=[]
    
    o =0
    for channel in dfc_temp_np :
        bl_r_brain_delta_temp = (dfc_temp_np[o][0]/ dfc_temp_np[o][4])*100
        bl_r_brain_delta.append(bl_r_brain_delta_temp)
        bl_r_brain_theta_temp = (dfc_temp_np[o][1]/ dfc_temp_np[o][4])*100
        bl_r_brain_theta.append(bl_r_brain_theta_temp)
        bl_r_brain_alpha_temp = (dfc_temp_np[o][2]/ dfc_temp_np[o][4])*100
        bl_r_brain_alpha.append(bl_r_brain_alpha_temp)
        bl_r_brain_beta_temp = (dfc_temp_np[o][3]/ dfc_temp_np[o][4])*100
        bl_r_brain_beta.append(bl_r_brain_beta_temp)
    
        r_brain_delta_temp = (dfc_temp_np[o][7]/ dfc_temp_np[o][11])*100
        r_brain_delta.append(r_brain_delta_temp)
        r_brain_theta_temp = (dfc_temp_np[o][8]/ dfc_temp_np[o][11])*100
        r_brain_theta.append(r_brain_theta_temp)
        r_brain_alpha_temp = (dfc_temp_np[o][9]/ dfc_temp_np[o][11])*100
        r_brain_alpha.append(r_brain_alpha_temp)
        r_brain_beta_temp = (dfc_temp_np[o][10]/ dfc_temp_np[o][11])*100
        r_brain_beta.append(r_brain_beta_temp)
        o=o+1

    dfc_temp['bl_r_brain_delta'] = bl_r_brain_delta
    dfc_temp['bl_r_brain_theta'] = bl_r_brain_theta
    dfc_temp['bl_r_brain_alpha'] = bl_r_brain_alpha
    dfc_temp['bl_r_brain_beta'] = bl_r_brain_beta
    dfc_temp['r_brain_delta'] = r_brain_delta
    dfc_temp['r_brain_theta'] = r_brain_theta
    dfc_temp['r_brain_alpha'] = r_brain_alpha
    dfc_temp['r_brain_beta'] = r_brain_beta
    fif_1_dfc_r_list.append(dfc_temp)
    p=p+1


In [94]:
fif_2_dfc_r_list[0]

,bl_delta_abp,bl_theta_abp,bl_alpha_abp,bl_beta_abp,bl_brain_abp,bl_full_abp,Channel,delta_abp,theta_abp,alpha_abp,...,full_abp,Channel,bl_r_brain_delta,bl_r_brain_theta,bl_r_brain_alpha,bl_r_brain_beta,r_brain_delta,r_brain_theta,r_brain_alpha,r_brain_beta
0,3.08e-11,2.02e-11,8.69e-11,1.55e-11,1.54e-10,1.62e-10,AF3,7.42e-11,2.63e-11,2.20e-11,...,1.69e-10,AF3,2.00e+01,1.31e+01,5.64e+01,1.01e+01,5.08e+01,1.80e+01,1.51e+01,1.60e+01
1,9.04e-12,5.96e-12,2.58e-11,4.54e-12,4.55e-11,4.80e-11,F7,2.17e-11,7.75e-12,6.49e-12,...,4.95e-11,F7,1.99e+01,1.31e+01,5.66e+01,9.98e+00,5.07e+01,1.81e+01,1.52e+01,1.60e+01
2,2.22e-11,1.45e-11,6.26e-11,1.10e-11,1.11e-10,1.19e-10,F3,4.87e-11,1.97e-11,1.55e-11,...,1.18e-10,F3,2.00e+01,1.31e+01,5.65e+01,9.97e+00,4.88e+01,1.98e+01,1.55e+01,1.59e+01
3,1.95e-11,1.26e-11,5.30e-11,9.96e-12,9.54e-11,1.01e-10,FC5,4.75e-11,1.64e-11,1.39e-11,...,1.07e-10,FC5,2.04e+01,1.32e+01,5.55e+01,1.04e+01,5.12e+01,1.77e+01,1.49e+01,1.61e+01
4,1.13e-11,7.43e-12,3.22e-11,5.65e-12,5.68e-11,5.98e-11,FC6,2.70e-11,9.67e-12,8.08e-12,...,6.16e-11,FC6,1.98e+01,1.31e+01,5.67e+01,9.95e+00,5.07e+01,1.82e+01,1.52e+01,1.60e+01
5,6.18e-11,4.00e-11,1.69e-10,3.15e-11,3.04e-10,3.21e-10,F4,1.51e-10,5.23e-11,4.41e-11,...,3.39e-10,F4,2.03e+01,1.32e+01,5.57e+01,1.04e+01,5.11e+01,1.78e+01,1.50e+01,1.61e+01
6,2.30e-11,1.52e-11,6.60e-11,1.15e-11,1.16e-10,1.23e-10,F8,5.51e-11,1.98e-11,1.65e-11,...,1.26e-10,F8,1.98e+01,1.31e+01,5.68e+01,9.93e+00,5.06e+01,1.82e+01,1.52e+01,1.60e+01
7,3.30e-11,2.17e-11,9.42e-11,1.65e-11,1.66e-10,1.75e-10,AF4,7.90e-11,2.83e-11,2.36e-11,...,1.80e-10,AF4,1.98e+01,1.31e+01,5.67e+01,9.95e+00,5.07e+01,1.81e+01,1.52e+01,1.60e+01


In [96]:
# fif_2_dfc_r_list_np = np.array(fif_2_dfc_r_list)
# fif_2_af3_all_participants = fif_2_dfc_r_list_np[:,0,:]
# fif_2_f7_all_participants = fif_2_dfc_r_list_np[:,1,:]
# fif_2_f3_all_participants = fif_2_dfc_r_list_np[:,2,:]
# fif_2_fc5_all_participants = fif_2_dfc_r_list_np[:,3,:]
# fif_2_fc6_all_participants = fif_2_dfc_r_list_np[:,4,:]
# fif_2_f4_all_participants = fif_2_dfc_r_list_np[:,5,:]
# fif_2_f8_all_participants = fif_2_dfc_r_list_np[:,6,:]
# fif_2_af4_all_participants = fif_2_dfc_r_list_np[:,7,:]

In [103]:
fif_1_dfc_r_list_np = np.array(fif_1_dfc_r_list)
fif_1_af3_all_participants = fif_1_dfc_r_list_np[:,0,:]
fif_1_f7_all_participants = fif_1_dfc_r_list_np[:,1,:]
fif_1_f3_all_participants = fif_1_dfc_r_list_np[:,2,:]
fif_1_fc5_all_participants = fif_1_dfc_r_list_np[:,3,:]
fif_1_fc6_all_participants = fif_1_dfc_r_list_np[:,4,:]
fif_1_f4_all_participants = fif_1_dfc_r_list_np[:,5,:]
fif_1_f8_all_participants = fif_1_dfc_r_list_np[:,6,:]
fif_1_af4_all_participants = fif_1_dfc_r_list_np[:,7,:]

In [80]:
# dfc_r_list_np = np.array(dfc_r_list)
# af3_all_participants = dfc_r_list_np[:,0,:]
# f7_all_participants = dfc_r_list_np[:,1,:]
# f3_all_participants = dfc_r_list_np[:,2,:]
# fc5_all_participants = dfc_r_list_np[:,3,:]
# fc6_all_participants = dfc_r_list_np[:,4,:]
# f4_all_participants = dfc_r_list_np[:,5,:]
# f8_all_participants = dfc_r_list_np[:,6,:]
# af4_all_participants = dfc_r_list_np[:,7,:]

In [97]:
data_columns = [ 
    'bl_delta_abp',
    'bl_theta_abp',
    'bl_alpha_abp',
    'bl_beta_abp',
    'bl_brain_abp',
    'bl_full_abp',
    'Channel',
    'delta_abp',
    'theta_abp',
    'alpha_abp',
    'beta_abp',
    'brain_abp',
    'full_abp',
    'Channel',
    'bl_r_brain_delta',
    'bl_r_brain_theta',
    'bl_r_brain_alpha',
    'bl_r_brain_beta',
    'r_brain_delta',
    'r_brain_theta',
    'r_brain_alpha',
    'r_brain_beta']

In [87]:
# df_all_af3 = pd.DataFrame(af3_all_participants, columns=data_columns)
# df_all_af3.to_csv('/Users/apple/Desktop/New ABP Results/Task 3/Channelwise_all participants/AF3_consolidated.csv')
# df_all_f7 = pd.DataFrame(f7_all_participants, columns=data_columns)
# df_all_f7.to_csv('/Users/apple/Desktop/New ABP Results/Task 3/Channelwise_all participants/F7_consolidated.csv')
# df_all_f3 = pd.DataFrame(f3_all_participants, columns=data_columns)
# df_all_f3.to_csv('/Users/apple/Desktop/New ABP Results/Task 3/Channelwise_all participants/F3_consolidated.csv')
# df_all_fc5 = pd.DataFrame(fc5_all_participants, columns=data_columns)
# df_all_fc5.to_csv('/Users/apple/Desktop/New ABP Results/Task 3/Channelwise_all participants/FC5_consolidated.csv')
# df_all_fc6 = pd.DataFrame(fc6_all_participants, columns=data_columns)
# df_all_fc6.to_csv('/Users/apple/Desktop/New ABP Results/Task 3/Channelwise_all participants/FC6_consolidated.csv')
# df_all_f4 = pd.DataFrame(f4_all_participants, columns=data_columns)
# df_all_f4.to_csv('/Users/apple/Desktop/New ABP Results/Task 3/Channelwise_all participants/F4_consolidated.csv')
# df_all_f8 = pd.DataFrame(f8_all_participants, columns=data_columns)
# df_all_f8.to_csv('/Users/apple/Desktop/New ABP Results/Task 3/Channelwise_all participants/F8_consolidated.csv')
# df_all_af4 = pd.DataFrame(af4_all_participants, columns=data_columns)
# df_all_af4.to_csv('/Users/apple/Desktop/New ABP Results/Task 3/Channelwise_all participants/AF4_consolidated.csv')

In [105]:
# fif_2_df_all_af3 = pd.DataFrame(fif_2_af3_all_participants, columns=data_columns)
# fif_2_df_all_af3.to_csv('/Users/apple/Desktop/New ABP Results/Task 2/Channelwise_all participants/AF3_consolidated.csv')
# fif_2_df_all_f7 = pd.DataFrame(fif_2_f7_all_participants, columns=data_columns)
# fif_2_df_all_f7.to_csv('/Users/apple/Desktop/New ABP Results/Task 2/Channelwise_all participants/F7_consolidated.csv')
# fif_2_df_all_f3 = pd.DataFrame(fif_2_f3_all_participants, columns=data_columns)
# fif_2_df_all_f3.to_csv('/Users/apple/Desktop/New ABP Results/Task 2/Channelwise_all participants/F3_consolidated.csv')
# fif_2_df_all_fc5 = pd.DataFrame(fif_2_fc5_all_participants, columns=data_columns)
# fif_2_df_all_fc5.to_csv('/Users/apple/Desktop/New ABP Results/Task 2/Channelwise_all participants/FC5_consolidated.csv')
# fif_2_df_all_fc6 = pd.DataFrame(fif_2_fc6_all_participants, columns=data_columns)
# fif_2_df_all_fc6.to_csv('/Users/apple/Desktop/New ABP Results/Task 2/Channelwise_all participants/FC6_consolidated.csv')
# fif_2_df_all_f4 = pd.DataFrame(fif_2_f4_all_participants, columns=data_columns)
# fif_2_df_all_f4.to_csv('/Users/apple/Desktop/New ABP Results/Task 2/Channelwise_all participants/F4_consolidated.csv')
# fif_2_df_all_f8 = pd.DataFrame(fif_2_f8_all_participants, columns=data_columns)
# fif_2_df_all_f8.to_csv('/Users/apple/Desktop/New ABP Results/Task 2/Channelwise_all participants/F8_consolidated.csv')
# fif_2_df_all_af4 = pd.DataFrame(fif_2_af4_all_participants, columns=data_columns)
# fif_2_df_all_af4.to_csv('/Users/apple/Desktop/New ABP Results/Task 2/Channelwise_all participants/AF4_consolidated.csv')

In [106]:
fif_1_df_all_af3 = pd.DataFrame(fif_1_af3_all_participants, columns=data_columns)
fif_1_df_all_af3.to_csv('/Users/apple/Desktop/New ABP Results/Task 1/Channelwise_all participants/AF3_consolidated.csv')
fif_1_df_all_f7 = pd.DataFrame(fif_1_f7_all_participants, columns=data_columns)
fif_1_df_all_f7.to_csv('/Users/apple/Desktop/New ABP Results/Task 1/Channelwise_all participants/F7_consolidated.csv')
fif_1_df_all_f3 = pd.DataFrame(fif_1_f3_all_participants, columns=data_columns)
fif_1_df_all_f3.to_csv('/Users/apple/Desktop/New ABP Results/Task 1/Channelwise_all participants/F3_consolidated.csv')
fif_1_df_all_fc5 = pd.DataFrame(fif_1_fc5_all_participants, columns=data_columns)
fif_1_df_all_fc5.to_csv('/Users/apple/Desktop/New ABP Results/Task 1/Channelwise_all participants/FC5_consolidated.csv')
fif_1_df_all_fc6 = pd.DataFrame(fif_1_fc6_all_participants, columns=data_columns)
fif_1_df_all_fc6.to_csv('/Users/apple/Desktop/New ABP Results/Task 1/Channelwise_all participants/FC6_consolidated.csv')
fif_1_df_all_f4 = pd.DataFrame(fif_1_f4_all_participants, columns=data_columns)
fif_1_df_all_f4.to_csv('/Users/apple/Desktop/New ABP Results/Task 1/Channelwise_all participants/F4_consolidated.csv')
fif_1_df_all_f8 = pd.DataFrame(fif_1_f8_all_participants, columns=data_columns)
fif_1_df_all_f8.to_csv('/Users/apple/Desktop/New ABP Results/Task 1/Channelwise_all participants/F8_consolidated.csv')
fif_1_df_all_af4 = pd.DataFrame(fif_1_af4_all_participants, columns=data_columns)
fif_1_df_all_af4.to_csv('/Users/apple/Desktop/New ABP Results/Task 1/Channelwise_all participants/AF4_consolidated.csv')

In [60]:
a = np.array([[[1,2,3],[4,5,6],[7,8,9]],[[10,11,12],[13,14,15],[16,17,18]],[[19,20,21],[22,23,24],[25,26,27]]])

In [67]:
a[:,0,:]

array([[ 1,  2,  3],
       [10, 11, 12],
       [19, 20, 21]])

In [19]:
#Export the values

bl_path = '/Users/apple/Desktop/New ABP Results/Task 3/Baseline/'
sksig_path = '/Users/apple/Desktop/New ABP Results/Task 3/Sketch Signal/'


k=0
for item in fif_3_bl_participantwise_abp_df : 
    item.to_csv(bl_path + participant_list[k] + '_task3_bl.csv')
    k=k+1

l=0
for item in fif_3_participantwise_abp_df : 
    item.to_csv(sksig_path + participant_list[l] + '_task3.csv')
    l=l+1




In [25]:
concat_path = '/Users/apple/Desktop/New ABP Results/Task 3/Concatenated/'
n=0
for item in concatenated_bl_abp_list : 
    item.to_csv(concat_path + participant_list[n] + '_task3_bl_abp.csv')
    n=n+1

In [56]:
rel_path = '/Users/apple/Desktop/New ABP Results/Task 3/Relative Power/'
q=0
for item in dfc_r_list : 
    item.to_csv(rel_path + participant_list[q] + '_task3_rel.csv')
    q=q+1